In [9]:
import sys



sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
import os

os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')
from sklearn import preprocessing
from sem_covid.services.store_registry import StoreRegistry
from pycaret.classification import *
import mlflow
from sem_covid import config


In [3]:
SIMPLE_CLASS_COLUMNS = ['category', 'subcategory', 'type_of_measure', 'target_groups',
                        'actors']

In [2]:
es_store = StoreRegistry.es_index_store()
df = es_store.get_dataframe(config.PWDB_ELASTIC_SEARCH_INDEX_NAME)

100% (1288 of 1288) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [4]:
pwdb_dataframe = df
pwdb_dataframe['target_groups'] = pwdb_dataframe['target_groups'].apply(lambda x: "|".join(x))
pwdb_dataframe['actors'] = pwdb_dataframe['actors'].apply(lambda x: "|".join(x))

In [36]:
pwdb_dataframe['actors_label'] = pwdb_dataframe['actors']

In [37]:
pwdb_dataframe[['actors','actors_label']]

,actors,actors_label
_id,,
adc5c75937bc7f7198f534d08b85bd50c9521bfd3f319a090932b5d0bae54de0,National government|Trade unions|Employers' or...,National government|Trade unions|Employers' or...
2372d71eb9ad6e6a70982e02bbe802db004ed49d91b2264c0a2e8e41571002cc,National government|Social insurance,National government|Social insurance
8735e268191e9e5cbd3d2a44ca53d297e31746b5f1e24b941db6225a25848353,National government|Local / regional governmen...,National government|Local / regional governmen...
18bcd22116c46919e03a3345f793c3859855227ac942e69dd13cbfcd588e1044,National government|Company / Companies,National government|Company / Companies
b94d8aa95fbdeb1bb832b01fbe5d6e9bf9fc36fceb14f7ba370a963f472fe35b,National government,National government
...,...,...
cb014a456b14c3621dd318a12e611f70c2a9636be9fe181072bd4bf5917a40fa,National government|Public employment service|...,National government|Public employment service|...
d233b17dc2b98f14269c2b22be78d93ec5ccf2a0013b86f09175c69353c5800b,National government|Social partners jointly,National government|Social partners jointly
77d7e3c52aaf78bdfb1a1667641db1293bbff862440c547fc3f6e8ab4fbd0d4e,National government,National government


In [38]:
le = preprocessing.LabelEncoder()
le.fit(pwdb_dataframe['actors'])

LabelEncoder()

In [40]:
pwdb_dataframe['actors'] = le.transform(pwdb_dataframe['actors'])

In [43]:
pwdb_dataframe['actors']= le.inverse_transform(pwdb_dataframe['actors'])

In [44]:
pwdb_dataframe['actors']

_id
adc5c75937bc7f7198f534d08b85bd50c9521bfd3f319a090932b5d0bae54de0    National government|Trade unions|Employers' or...
2372d71eb9ad6e6a70982e02bbe802db004ed49d91b2264c0a2e8e41571002cc                 National government|Social insurance
8735e268191e9e5cbd3d2a44ca53d297e31746b5f1e24b941db6225a25848353    National government|Local / regional governmen...
18bcd22116c46919e03a3345f793c3859855227ac942e69dd13cbfcd588e1044              National government|Company / Companies
b94d8aa95fbdeb1bb832b01fbe5d6e9bf9fc36fceb14f7ba370a963f472fe35b                                  National government
                                                                                          ...                        
cb014a456b14c3621dd318a12e611f70c2a9636be9fe181072bd4bf5917a40fa    National government|Public employment service|...
d233b17dc2b98f14269c2b22be78d93ec5ccf2a0013b86f09175c69353c5800b          National government|Social partners jointly
77d7e3c52aaf78bdfb1a1667641db1293bbff862440c547fc3f6

In [20]:
le.classes_

array(['Company / Companies',
       'Company / Companies|Local / regional government',
       'Company / Companies|Local / regional government|Public employment service',
       'Company / Companies|Other social actors (e.g. NGOs)',
       'Company / Companies|Other social actors (e.g. NGOs)|Local / regional government',
       'Company / Companies|Other social actors (e.g. NGOs)|Public support service providers',
       'Company / Companies|Public employment service',
       'Company / Companies|Public support service providers',
       'Company / Companies|Social insurance', 'EU (Council, EC, EP)',
       'EU level social partners', "Employers' organisations",
       "Employers' organisations|Company / Companies",
       "Employers' organisations|Company / Companies|Local / regional government",
       "Employers' organisations|Company / Companies|Other social actors (e.g. NGOs)",
       "Employers' organisations|Company / Companies|Social insurance",
       "Employers' organisation

In [18]:
tmp_classes = pd.DataFrame(le.classes_)

In [30]:
tmp_classes[0].values
tmp_transformed = pd.DataFrame(le.transform(tmp_classes[0].values))

In [33]:
tmp_transformed[1] = tmp_transformed[0].apply(lambda x: tmp_classes[0].values[x])

In [34]:
tmp_transformed

,0,1
0,0,Company / Companies
1,1,Company / Companies|Local / regional government
2,2,Company / Companies|Local / regional governmen...
3,3,Company / Companies|Other social actors (e.g. ...
4,4,Company / Companies|Other social actors (e.g. ...
5,5,Company / Companies|Other social actors (e.g. ...
6,6,Company / Companies|Public employment service
7,7,Company / Companies|Public support service pro...
8,8,Company / Companies|Social insurance
9,9,"EU (Council, EC, EP)"


In [ ]:
for column in SIMPLE_CLASS_COLUMNS:
    le = preprocessing.LabelEncoder()
    le.fit(pwdb_dataframe[column])
    pwdb_dataframe[column + '_label'] = le.transform(pwdb_dataframe[column])